## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-23-10-49-56 +0000,nyt,Russia Bombards Ukraine Hours After Zelensky C...,https://www.nytimes.com/2025/12/23/world/europ...
1,2025-12-23-10-32-38 +0000,nypost,Nearly 100 Minnesota mayors warn state fiscal ...,https://nypost.com/2025/12/23/us-news/98-minne...
2,2025-12-23-10-30-00 +0000,wsj,A Small Nebraska Town Is Reeling From the Exit...,https://www.wsj.com/us-news/a-small-nebraska-t...
3,2025-12-23-10-30-00 +0000,wsj,Car Payments Now Average More Than $750 a Mont...,https://www.wsj.com/business/autos/car-payment...
4,2025-12-23-10-28-34 +0000,bbc,Gunmen abduct 28 Muslim travellers in central ...,https://www.bbc.com/news/articles/czj0ll7ppewo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
52,trump,43
71,new,15
161,up,11
153,china,10
149,christmas,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
48,2025-12-23-05-53-43 +0000,nyt,Trump Announces New ‘Trump Class’ of Warships,https://www.nytimes.com/2025/12/22/us/politics...,108
115,2025-12-23-00-25-00 +0000,wsj,President Trump said he wants to pressure Amer...,https://www.wsj.com/politics/policy/trump-targ...,97
232,2025-12-22-16-52-46 +0000,nypost,Trump admin scraps 5 offshore wind farms — inc...,https://nypost.com/2025/12/22/us-news/trump-ad...,89
196,2025-12-22-19-16-47 +0000,nypost,Bill Clinton accuses Trump’s DOJ of cover-up i...,https://nypost.com/2025/12/22/us-news/bill-cli...,87
158,2025-12-22-21-55-00 +0000,wsj,A coalition of Democratic state attorneys gene...,https://www.wsj.com/politics/policy/democratic...,76


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
48,108,2025-12-23-05-53-43 +0000,nyt,Trump Announces New ‘Trump Class’ of Warships,https://www.nytimes.com/2025/12/22/us/politics...
123,55,2025-12-22-23-38-39 +0000,nypost,Warner Bros. Discovery to review Larry Ellison...,https://nypost.com/2025/12/22/media/warner-bro...
244,53,2025-12-22-15-39-36 +0000,wapo,Car bomb in Moscow kills senior Russian genera...,https://www.washingtonpost.com/world/2025/12/2...
175,44,2025-12-22-20-39-00 +0000,wsj,Former Vice President Mike Pence’s political g...,https://www.wsj.com/politics/elections/top-her...
196,42,2025-12-22-19-16-47 +0000,nypost,Bill Clinton accuses Trump’s DOJ of cover-up i...,https://nypost.com/2025/12/22/us-news/bill-cli...
62,38,2025-12-23-03-41-55 +0000,cbc,60 Minutes prison segment postponed by CBS bri...,https://www.cbc.ca/news/canada/cbs-60-minutes-...
65,36,2025-12-23-03-30-06 +0000,nypost,JD Vance trains with Navy SEALs in California:...,https://nypost.com/2025/12/22/us-news/jd-vance...
61,36,2025-12-23-03-56-42 +0000,bbc,Brown University places its police chief on le...,https://www.bbc.com/news/articles/cy0p7d74ywgo...
98,32,2025-12-23-01-44-54 +0000,nypost,At least 2 people killed in Mexican Navy plane...,https://nypost.com/2025/12/22/us-news/at-least...
232,31,2025-12-22-16-52-46 +0000,nypost,Trump admin scraps 5 offshore wind farms — inc...,https://nypost.com/2025/12/22/us-news/trump-ad...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
